<a href="https://colab.research.google.com/github/harish-rajan/colab/blob/main/qwen_fin_tuning_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch transformers bitsandbytes trl peft datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch, os, multiprocessing
from datasets import load_dataset
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    set_seed
)
from trl import SFTTrainer, SFTConfig

set_seed(2)

In [3]:
!pip install flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash_attn: filename=flash_attn-2.7.2.post1-cp310-cp310-linux_x86_64.whl size=190160474 sha256=0b454d9e650bfc437cc71335080172a5d05f51eab355636c9d5b7321fec7318e
  Stored in directory: /root/.cache/pip/wheels/da/ec/5b/b2c37a8e4f755ad82492a822463bca0817f0e0e11de874b550
Successfully built flash_attn


In [4]:
#use bf16 and FlashAttention if supported
if torch.cuda.is_bf16_supported():
  os.system('pip install flash_attn')
  compute_dtype = torch.bfloat16
  attn_implementation = 'flash_attention_2'
else:
  compute_dtype = torch.float16
  attn_implementation = 'sdpa'

In [5]:
model_name = "Qwen/Qwen2.5-7B"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, device_map="auto")
tokenizer.pad_token = "<|im_end|>"
tokenizer.pad_token_id = 151645
tokenizer.padding_side = 'left'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [6]:
ds = load_dataset("json", data_files="trl_format_1.json")

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
ds_ = ds['train'].train_test_split(test_size=0.2, seed=2)

In [8]:
def process(row):
    row["text"] = row["text"]  + tokenizer.eos_token
    return row

In [9]:
ds = ds.map(
    process,
    #num_proc= multiprocessing.cpu_count(),
    load_from_cache_file=False,
)

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [11]:
model = AutoModelForCausalLM.from_pretrained(
          model_name, device_map={"": 0}, attn_implementation=attn_implementation
)


config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [12]:
model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={'use_reentrant':True})

In [13]:
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=16,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]
)

In [14]:
training_arguments = SFTConfig(
        output_dir="./Qwen2.5_7B_LoRA",
        eval_strategy="steps",
        do_eval=True,
        optim="paged_adamw_8bit",
        #optim="adamw_torch",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=16,
        per_device_eval_batch_size=2,
        log_level="debug",
        save_strategy="epoch",
        logging_steps=25,
        learning_rate=1e-4,
        fp16 = False,
        bf16 = True,
        eval_steps=25,
        num_train_epochs=1,
        warmup_ratio=0.1,
        lr_scheduler_type="linear",
        dataset_text_field="text",
        max_seq_length=512,
        #no_cuda=True  # Prevent using CUDA backend
)

In [15]:
trainer = SFTTrainer(
        model=model,
        train_dataset=ds_['train'],
        eval_dataset=ds_['test'],
        peft_config=peft_config,
        tokenizer=tokenizer,
        args=training_arguments,
)

<ipython-input-15-32b976b53e22>:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/9 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(
Using auto half precision backend


In [16]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")


GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.564 GB.
28.73 GB of memory reserved.


In [17]:
trainer_t = trainer.train()

Currently training with a batch size of: 2
***** Running training *****
  Num examples = 9
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 16
  Total optimization steps = 1
  Number of trainable parameters = 40,370,176
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.


Step,Training Loss,Validation Loss


Saving model checkpoint to ./Qwen2.5_7B_LoRA/checkpoint-1
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2.5-7B/snapshots/d149729398750b98c0af14eb82c78cfe92750796/config.json
Model config Qwen2Config {
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 3584,
  "initializer_range": 0.02,
  "intermediate_size": 18944,
  "max_position_embeddings": 131072,
  "max_window_layers": 28,
  "model_type": "qwen2",
  "num_attention_heads": 28,
  "num_hidden_layers": 28,
  "num_key_value_heads": 4,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.47.1",
  "use_cache": true,
  "use_mrope": false,
  "use_sliding_window": false,
  "vocab_size": 152064
}

tokenizer config file saved in ./Qwen2

In [20]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_trainer= round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
trainer_percentage = round(used_memory_for_trainer/max_memory*100, 3)
print(f"{trainer_t.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_t.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_trainer} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {trainer_percentage} %.")
print("-----")


110.2597 seconds used for training.
1.84 minutes used for training.
Peak reserved memory = 33.314 GB.
Peak reserved memory for training = 4.584 GB.
Peak reserved memory % of max memory = 84.203 %.
Peak reserved memory for training % of max memory = 11.586 %.
-----


In [27]:
MODEL_SAVE_FOLDER_NAME = "qwen-2.5-lora"

import os
os.makedirs(MODEL_SAVE_FOLDER_NAME, exist_ok=True)

trainer.save_model(MODEL_SAVE_FOLDER_NAME+"/qwen2.5_lora_model")
tokenizer.save_pretrained(MODEL_SAVE_FOLDER_NAME+"/qwen2.5_lora_model")

Saving model checkpoint to qwen-2.5-lora/qwen2.5_lora_model
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2.5-7B/snapshots/d149729398750b98c0af14eb82c78cfe92750796/config.json
Model config Qwen2Config {
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 3584,
  "initializer_range": 0.02,
  "intermediate_size": 18944,
  "max_position_embeddings": 131072,
  "max_window_layers": 28,
  "model_type": "qwen2",
  "num_attention_heads": 28,
  "num_hidden_layers": 28,
  "num_key_value_heads": 4,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.47.1",
  "use_cache": true,
  "use_mrope": false,
  "use_sliding_window": false,
  "vocab_size": 152064
}

tokenizer config file saved in qwen-

('qwen-2.5-lora/qwen2.5_lora_model/tokenizer_config.json',
 'qwen-2.5-lora/qwen2.5_lora_model/special_tokens_map.json',
 'qwen-2.5-lora/qwen2.5_lora_model/vocab.json',
 'qwen-2.5-lora/qwen2.5_lora_model/merges.txt',
 'qwen-2.5-lora/qwen2.5_lora_model/added_tokens.json',
 'qwen-2.5-lora/qwen2.5_lora_model/tokenizer.json')

In [ ]:
from peft import PeftModel

# only saves the incremental PEFT weights(adapter_model.bin) that were trained, meaning it is super  efficient
#trainer_t.model.save_pretrained(MODEL_SAVE_FOLDER_NAME)

# save the full model and the training arguments
trainer_t.save_model(MODEL_SAVE_FOLDER_NAME)
trainer_t.model.config.save_pretrained(MODEL_SAVE_FOLDER_NAME)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=compute_dtype)
model = PeftModel.from_pretrained(model, adapter)
model = model.merge_and_unload()
model.save_pretrained("./naive_merge/", safe_serialization=True)

In [25]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=607788d5860fcdb87f0032d5ed6a93c33024eb0312d97d4a4f6835bb44fad80f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [29]:
from rouge_score import rouge_scorer

# Reference and generated text
reference = "Generate an unit test code for the following code written using Apache Flink. \n package org.apache.flink.state.api.input;\n\nimport org.apache.flink.annotation.Internal;\nimport org.apache.flink.api.common.ExecutionConfig;\nimport org.apache.flink.api.common.functions.RuntimeContext;\nimport org.apache.flink.api.common.io.DefaultInputSplitAssigner;\nimport org.apache.flink.api.common.io.RichInputFormat;\nimport org.apache.flink.api.common.io.statistics.BaseStatistics;\nimport org.apache.flink.api.java.tuple.Tuple2;\nimport org.apache.flink.configuration.Configuration;\nimport org.apache.flink.core.fs.CloseableRegistry;\nimport org.apache.flink.core.io.InputSplitAssigner;\nimport org.apache.flink.runtime.checkpoint.OperatorState;\nimport org.apache.flink.runtime.checkpoint.StateAssignmentOperation;\nimport org.apache.flink.runtime.state.AbstractKeyedStateBackend;\nimport org.apache.flink.runtime.state.DefaultKeyedStateStore;\nimport org.apache.flink.runtime.state.KeyGroupRange;\nimport org.apache.flink.runtime.state.KeyedStateHandle;\nimport org.apache.flink.runtime.state.StateBackend;\nimport org.apache.flink.state.api.functions.KeyedStateReaderFunction;\nimport org.apache.flink.state.api.input.operator.StateReaderOperator;\nimport org.apache.flink.state.api.input.splits.KeyGroupRangeInputSplit;\nimport org.apache.flink.state.api.runtime.SavepointRuntimeContext;\nimport org.apache.flink.streaming.api.operators.InternalTimeServiceManager;\nimport org.apache.flink.streaming.api.operators.StreamOperatorStateContext;\nimport org.apache.flink.util.CloseableIterator;\nimport org.apache.flink.util.CollectionUtil;\nimport org.apache.flink.util.IOUtils;\nimport org.apache.flink.util.Preconditions;\nimport org.apache.flink.util.SerializedValue;\n\nimport org.slf4j.Logger;\nimport org.slf4j.LoggerFactory;\n\nimport javax.annotation.Nonnull;\nimport javax.annotation.Nullable;\n\nimport java.io.IOException;\nimport java.util.Comparator;\nimport java.util.List;\n\n/**\n * Input format for reading partitioned state.\n *\n * @param <K> The type of the key.\n * @param <OUT> The type of the output of the {@link KeyedStateReaderFunction}.\n */\n@Internal\npublic class KeyedStateInputFormat<K, N, OUT>\n        extends RichInputFormat<OUT, KeyGroupRangeInputSplit> {\n\n    private static final long serialVersionUID = 8230460226049597182L;\n\n    private static final Logger LOG = LoggerFactory.getLogger(KeyedStateInputFormat.class);\n\n    private final OperatorState operatorState;\n\n    @Nullable private final StateBackend stateBackend;\n\n    private final Configuration configuration;\n\n    private final StateReaderOperator<?, K, N, OUT> operator;\n\n    private final SerializedValue<ExecutionConfig> serializedExecutionConfig;\n\n    private transient CloseableRegistry registry;\n\n    private transient BufferingCollector<OUT> out;\n\n    private transient CloseableIterator<Tuple2<K, N>> keysAndNamespaces;\n\n    /**\n     * Creates an input format for reading partitioned state from an operator in a savepoint.\n     *\n     * @param operatorState The state to be queried.\n     * @param stateBackend The state backed used to snapshot the operator.\n     * @param configuration The underlying Flink configuration used to configure the state backend.\n     */\n    public KeyedStateInputFormat(\n            OperatorState operatorState,\n            @Nullable StateBackend stateBackend,\n            Configuration configuration,\n            StateReaderOperator<?, K, N, OUT> operator,\n            ExecutionConfig executionConfig)\n            throws IOException {\n        Preconditions.checkNotNull(operatorState, \\\"The operator state cannot be null\\\");\n        Preconditions.checkNotNull(configuration, \\\"The configuration cannot be null\\\");\n        Preconditions.checkNotNull(operator, \\\"The operator cannot be null\\\");\n        Preconditions.checkNotNull(executionConfig, \\\"The executionConfig cannot be null\\\");\n\n        this.operatorState = operatorState;\n        this.stateBackend = stateBackend;\n        // Eagerly deep copy the configuration object\n        // otherwise there will be undefined behavior\n        // when executing pipelines with multiple input formats\n        this.configuration = new Configuration(configuration);\n        this.operator = operator;\n        this.serializedExecutionConfig = new SerializedValue<>(executionConfig);\n    }\n\n    @Override\n    public void configure(Configuration parameters) {}\n\n    @Override\n    public InputSplitAssigner getInputSplitAssigner(KeyGroupRangeInputSplit[] inputSplits) {\n        return new DefaultInputSplitAssigner(inputSplits);\n    }\n\n    @Override\n    public BaseStatistics getStatistics(BaseStatistics cachedStatistics) {\n        return cachedStatistics;\n    }\n\n    @Override\n    public KeyGroupRangeInputSplit[] createInputSplits(int minNumSplits) throws IOException {\n        final int maxParallelism = operatorState.getMaxParallelism();\n\n        final List<KeyGroupRange> keyGroups = sortedKeyGroupRanges(minNumSplits, maxParallelism);\n\n        return CollectionUtil.mapWithIndex(\n                        keyGroups,\n                        (keyGroupRange, index) ->\n                                createKeyGroupRangeInputSplit(\n                                        operatorState, maxParallelism, keyGroupRange, index))\n                .toArray(KeyGroupRangeInputSplit[]::new);\n    }\n\n    @Override\n    public void openInputFormat() {\n        out = new BufferingCollector<>();\n    }\n\n    @Override\n    @SuppressWarnings(\\\"unchecked\\\")\n    public void open(KeyGroupRangeInputSplit split) throws IOException {\n        registry = new CloseableRegistry();\n\n        RuntimeContext runtimeContext = getRuntimeContext();\n        ExecutionConfig executionConfig;\n        try {\n            executionConfig =\n                    serializedExecutionConfig.deserializeValue(\n                            runtimeContext.getUserCodeClassLoader());\n        } catch (ClassNotFoundException e) {\n            throw new RuntimeException(\\\"Could not deserialize ExecutionConfig.\\\", e);\n        }\n        final StreamOperatorStateContext context =\n                new StreamOperatorContextBuilder(\n                                runtimeContext,\n                                configuration,\n                                operatorState,\n                                split,\n                                registry,\n                                stateBackend,\n                                executionConfig)\n                        .withMaxParallelism(split.getNumKeyGroups())\n                        .withKey(operator, runtimeContext.createSerializer(operator.getKeyType()))\n                        .build(LOG);\n\n        AbstractKeyedStateBackend<K> keyedStateBackend =\n                (AbstractKeyedStateBackend<K>) context.keyedStateBackend();\n\n        final DefaultKeyedStateStore keyedStateStore =\n                new DefaultKeyedStateStore(keyedStateBackend, runtimeContext::createSerializer);\n        SavepointRuntimeContext ctx = new SavepointRuntimeContext(runtimeContext, keyedStateStore);\n\n        InternalTimeServiceManager<K> timeServiceManager =\n                (InternalTimeServiceManager<K>) context.internalTimerServiceManager();\n        try {\n            operator.setup(\n                    runtimeContext::createSerializer, keyedStateBackend, timeServiceManager, ctx);\n            operator.open();\n            keysAndNamespaces = operator.getKeysAndNamespaces(ctx);\n        } catch (Exception e) {\n            throw new IOException(\\\"Failed to restore timer state\\\", e);\n        }\n    }\n\n    @Override\n    public void close() throws IOException {\n        try {\n            IOUtils.closeQuietly(keysAndNamespaces);\n            IOUtils.closeQuietly(operator);\n            IOUtils.closeQuietly(registry);\n        } catch (Exception e) {\n            throw new IOException(\\\"Failed to close state backend\\\", e);\n        }\n    }\n\n    @Override\n    public boolean reachedEnd() {\n        return !out.hasNext() && !keysAndNamespaces.hasNext();\n    }\n\n    @Override\n    public OUT nextRecord(OUT reuse) throws IOException {\n        if (out.hasNext()) {\n            return out.next();\n        }\n\n        final Tuple2<K, N> keyAndNamespace = keysAndNamespaces.next();\n        operator.setCurrentKey(keyAndNamespace.f0);\n\n        try {\n            operator.processElement(keyAndNamespace.f0, keyAndNamespace.f1, out);\n        } catch (Exception e) {\n            throw new IOException(\n                    \\\"User defined function KeyedStateReaderFunction#readKey threw an exception\\\", e);\n        }\n\n        keysAndNamespaces.remove();\n\n        return out.next();\n    }\n\n    private static KeyGroupRangeInputSplit createKeyGroupRangeInputSplit(\n            OperatorState operatorState,\n            int maxParallelism,\n            KeyGroupRange keyGroupRange,\n            Integer index) {\n\n        final List<KeyedStateHandle> managedKeyedState =\n                StateAssignmentOperation.getManagedKeyedStateHandles(operatorState, keyGroupRange);\n        final List<KeyedStateHandle> rawKeyedState =\n                StateAssignmentOperation.getRawKeyedStateHandles(operatorState, keyGroupRange);\n\n        return new KeyGroupRangeInputSplit(managedKeyedState, rawKeyedState, maxParallelism, index);\n    }\n\n    @Nonnull\n    private static List<KeyGroupRange> sortedKeyGroupRanges(int minNumSplits, int maxParallelism) {\n        List<KeyGroupRange> keyGroups =\n                StateAssignmentOperation.createKeyGroupPartitions(\n                        maxParallelism, Math.min(minNumSplits, maxParallelism));\n\n        keyGroups.sort(Comparator.comparing(KeyGroupRange::getStartKeyGroup));\n        return keyGroups;\n    }\n}"
generated = "package org.apache.flink.state.api.input; \nimport org.apache.flink.api.common.ExecutionConfig;\nimport org.apache.flink.api.common.state.MapStateDescriptor;\nimport org.apache.flink.api.common.typeinfo.Types;\nimport org.apache.flink.api.java.tuple.Tuple2;\nimport org.apache.flink.configuration.Configuration;\nimport org.apache.flink.runtime.checkpoint.OperatorState;\nimport org.apache.flink.runtime.checkpoint.OperatorSubtaskState;\nimport org.apache.flink.state.api.input.splits.OperatorStateInputSplit;\nimport org.apache.flink.state.api.runtime.OperatorIDGenerator;\nimport org.apache.flink.streaming.api.functions.co.BroadcastProcessFunction;\nimport org.apache.flink.streaming.api.operators.co.CoBroadcastWithNonKeyedOperator;\nimport org.apache.flink.streaming.runtime.streamrecord.StreamRecord;\nimport org.apache.flink.streaming.util.MockStreamingRuntimeContext;\nimport org.apache.flink.streaming.util.TwoInputStreamOperatorTestHarness;\nimport org.apache.flink.util.Collector;\n\nimport org.junit.Assert;\nimport org.junit.Test;\n\nimport java.util.Collections;\nimport java.util.HashMap;\nimport java.util.Map;\n\n/** Test for operator broadcast state input format. */\npublic class BroadcastStateInputFormatTest {\n    private static MapStateDescriptor<Integer, Integer> descriptor =\n            new MapStateDescriptor<>(\\\"state\\\", Types.INT, Types.INT);\n\n    @Test\n    public void testReadBroadcastState() throws Exception {\n        try (TwoInputStreamOperatorTestHarness<Void, Integer, Void> testHarness =\n                getTestHarness()) {\n            testHarness.open();\n\n            testHarness.processElement2(new StreamRecord<>(1));\n            testHarness.processElement2(new StreamRecord<>(2));\n            testHarness.processElement2(new StreamRecord<>(3));\n\n            OperatorSubtaskState subtaskState = testHarness.snapshot(0, 0);\n            OperatorState state =\n                    new OperatorState(null, null, OperatorIDGenerator.fromUid(\\\"uid\\\"), 1, 4);\n            state.putState(0, subtaskState);\n\n            OperatorStateInputSplit split =\n                    new OperatorStateInputSplit(subtaskState.getManagedOperatorState(), 0);\n\n            BroadcastStateInputFormat<Integer, Integer> format =\n                    new BroadcastStateInputFormat<>(\n                            state, new Configuration(), null, descriptor, new ExecutionConfig());\n\n            format.setRuntimeContext(new MockStreamingRuntimeContext(false, 1, 0));\n            format.open(split);\n\n            Map<Integer, Integer> results = new HashMap<>(3);\n\n            while (!format.reachedEnd()) {\n                Tuple2<Integer, Integer> entry = format.nextRecord(null);\n                results.put(entry.f0, entry.f1);\n            }\n\n            Map<Integer, Integer> expected = new HashMap<>(3);\n            expected.put(1, 1);\n            expected.put(2, 2);\n            expected.put(3, 3);\n\n            Assert.assertEquals(\n                    \\\"Failed to read correct list state from state backend\\\", expected, results);\n        }\n    }\n\n    private TwoInputStreamOperatorTestHarness<Void, Integer, Void> getTestHarness()\n            throws Exception {\n        return new TwoInputStreamOperatorTestHarness<>(\n                new CoBroadcastWithNonKeyedOperator<>(\n                        new StatefulFunction(), Collections.singletonList(descriptor)));\n    }\n\n    static class StatefulFunction extends BroadcastProcessFunction<Void, Integer, Void> {\n\n        @Override\n        public void processElement(Void value, ReadOnlyContext ctx, Collector<Void> out) {}\n\n        @Override\n        public void processBroadcastElement(Integer value, Context ctx, Collector<Void> out)\n                throws Exception {\n            ctx.getBroadcastState(descriptor).put(value, value);\n        }\n    }\n "

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Calculate ROUGE scores
scores = scorer.score(reference, generated)

# Display ROUGE scores
print("ROUGE Scores:")
for key, value in scores.items():
    print(f"{key}: Precision: {value.precision:.4f}, Recall: {value.recall:.4f}, F1-Score: {value.fmeasure:.4f}")


ROUGE Scores:
rouge1: Precision: 0.5758, Recall: 0.2550, F1-Score: 0.3534
rouge2: Precision: 0.3042, Recall: 0.1345, F1-Score: 0.1865
rougeL: Precision: 0.4213, Recall: 0.1866, F1-Score: 0.2586


In [38]:
!pwd
!ls -r
!zip -r Qwen2.5_7B_LoRA.zip .


/content
wandb  trl_format_1.json  sample_data  qwen-2.5-lora  Qwen2.5_7B_LoRA
  adding: .config/ (stored 0%)
  adding: .config/.last_opt_in_prompt.yaml (stored 0%)
  adding: .config/.last_survey_prompt.yaml (stored 0%)
  adding: .config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db (deflated 97%)
  adding: .config/active_config (stored 0%)
  adding: .config/default_configs.db (deflated 98%)
  adding: .config/.last_update_check.json (deflated 23%)
  adding: .config/logs/ (stored 0%)
  adding: .config/logs/2025.01.02/ (stored 0%)
  adding: .config/logs/2025.01.02/14.18.37.807698.log (deflated 87%)
  adding: .config/logs/2025.01.02/14.18.38.781748.log (deflated 58%)
  adding: .config/logs/2025.01.02/14.18.49.781149.log (deflated 56%)
  adding: .config/logs/2025.01.02/14.18.27.587350.log (deflated 58%)
  adding: .config/logs/2025.01.02/14.18.49.176443.log (deflated 57%)
  adding: .config/logs/2025.01.02/14.18.07.073020.log (deflated 93%)
  adding: .config/config_sentinel 

In [ ]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer

model_name = "Qwen/Qwen2.5-7B"
adapters_name = 'timdettmers/guanaco-7b'

print(f"Starting to load the model {model_name} into memory")

m = AutoModelForCausalLM.from_pretrained(
    model_name,
    #load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map={"": 0}
)
m = PeftModel.from_pretrained(m, adapters_name)
m = m.merge_and_unload()
tok = LlamaTokenizer.from_pretrained(model_name)
tok.bos_token_id = 1

stop_token_ids = [0]

print(f"Successfully loaded the model {model_name} into memory")


In [40]:
from transformers import AutoModelForSequenceClassification

from peft import PeftModel

model_name = "Qwen/Qwen2.5-7B"

base_model = AutoModelForSequenceClassification.from_pretrained(model_name)

lora_model = PeftModel.from_pretrained(base_model, "qwen-2.5-lora/qwen2.5_lora_model")


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2.5-7B/snapshots/d149729398750b98c0af14eb82c78cfe92750796/config.json
Model config Qwen2Config {
  "_name_or_path": "Qwen/Qwen2.5-7B",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 3584,
  "initializer_range": 0.02,
  "intermediate_size": 18944,
  "max_position_embeddings": 131072,
  "max_window_layers": 28,
  "model_type": "qwen2",
  "num_attention_heads": 28,
  "num_hidden_layers": 28,
  "num_key_value_heads": 4,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.47.1",
  "use_cache": true,
  "use_mrope": false,
  "use_sliding_window": false,
  "vocab_size": 152064
}

loading weights file model.safetensors from cache at /root

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of the model checkpoint at Qwen/Qwen2.5-7B were not used when initializing Qwen2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing Qwen2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Qwen2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen2.5-7B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
MODEL_SAVE_FOLDER_NAME = "qwen-2.5-lora-final"

import os
os.makedirs(MODEL_SAVE_FOLDER_NAME, exist_ok=True)

lora_model.save_pretrained("qwen-2.5-lora-final/qwen2.5_lora_model_final")


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2.5-7B/snapshots/d149729398750b98c0af14eb82c78cfe92750796/config.json
Model config Qwen2Config {
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 3584,
  "initializer_range": 0.02,
  "intermediate_size": 18944,
  "max_position_embeddings": 131072,
  "max_window_layers": 28,
  "model_type": "qwen2",
  "num_attention_heads": 28,
  "num_hidden_layers": 28,
  "num_key_value_heads": 4,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.47.1",
  "use_cache": true,
  "use_mrope": false,
  "use_sliding_window": false,
  "vocab_size": 152064
}



In [42]:
!cd qwen-2.5-lora-final/qwen2.5_lora_model_final/ && zip -r qwen-2.5-lora-final.zip .


  adding: adapter_config.json (deflated 56%)
  adding: adapter_model.safetensors (deflated 59%)
  adding: README.md (deflated 66%)
